# ** Executando Pyspark em Colab **

Para executar o spark no Colab, precisamos primeiro instalar todas as dependências no ambiente Colab, ou seja, Apache Spark 2.3.2 com hadoop 2.7, Java 8 e Findspark para localizar o spark no sistema. A instalação das ferramentas pode ser realizada dentro do Notebook Jupyter do Colab. Uma observação importante é que, se você for novo no Spark, é melhor evitar a versão 2.4.0 do Spark, pois algumas pessoas já reclamaram do problema de compatibilidade com o python.
Siga as etapas para instalar as dependências:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

O sistema nÆo pode encontrar o caminho especificado.
'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
tar: Error opening archive: Failed to open 'spark-2.4.7-bin-hadoop2.7.tgz'


Agora que você instalou o Spark e o Java no Colab, é hora de definir o caminho do ambiente que permite executar o Pyspark em seu ambiente Colab. Defina a localização do Java e do Spark executando o seguinte código:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

Execute uma sessão Spark local para testar sua instalação:

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Parabéns! Seu Colab está pronto para executar o Pyspark. Vamos construir um modelo de regressão linear simples.

# Modelo de regressão linear


O modelo de regressão linear é uma das abordagens de aprendizado de máquina mais antigas e amplamente utilizadas, que assume uma relação entre variáveis dependentes e independentes. Por exemplo, um modelador pode querer prever a previsão da chuva com base na taxa de umidade. A regressão linear consiste na linha de melhor ajuste através dos pontos dispersos no gráfico e a linha de melhor ajuste é conhecida como linha de regressão.

O objetivo deste exercício é prever os preços da habitação pelas características dadas. Vamos prever os preços do conjunto de dados do Boston Housing considerando MEDV como a variável de saída e todas as outras variáveis como entrada.

Baixe o conjunto de dados [aqui] (https://github.com/asifahmed90/pyspark-ML-in-Colab/blob/master/BostonHousing.csv) e mantenha-o em algum lugar em seu computador. Carregue o conjunto de dados em seu diretório Colab a partir de seu sistema local:

In [2]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'termios'

Verifique se o conjunto de dados foi carregado corretamente no sistema com o seguinte comando

Agora que carregamos o conjunto de dados, podemos começar a analisar.
Para o nosso modelo de regressão linear, precisamos importar dois módulos do Pyspark, ou seja, Vector Assembler e Regressão Linear. Vector Assembler é um transformador que reúne todos os recursos em um vetor a partir de várias colunas que contêm o tipo duplo. Poderíamos ter usado StringIndexer se alguma de nossas colunas contivesse valores de string para convertê-la em valores numéricos. Felizmente, o conjunto de dados BostonHousing contém apenas valores duplos, então não precisamos nos preocupar com StringIndexer por enquanto.

In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

AnalysisException: Path does not exist: file:/C:/Users/ricar/git/curso_big_data_com_mapreduce_spark/03_Regressao_Linear/BostonHousing.csv;

Observe que usamos InferSchema dentro do read.csv mofule. InferSchema nos permite inferir automaticamente diferentes tipos de dados para cada coluna.

Vamos dar uma olhada no conjunto de dados para ver os tipos de dados de cada coluna:

In [ ]:
dataset.printSchema()

O próximo passo é converter todos os recursos de diferentes colunas em uma única coluna e vamos chamar essa nova coluna vetorial como 'Atributos' no outputCol.

In [ ]:
#Insira todos os recursos em uma coluna de vetor
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show()

Aqui, 'Atributos' estão nos recursos de entrada de todas as colunas e 'medv' é a coluna de destino.
Em seguida, devemos dividir os dados de treinamento e teste de acordo com nosso conjunto de dados (0,8 e 0,2 neste caso).

In [ ]:
#Dividir dados de treinamento e teste
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

# Aprenda a se ajustar ao modelo do conjunto de treinamento
regressor = regressor.fit(train_data)

#Para prever os preços no conjunto de teste
pred = regressor.evaluate(test_data)

#Predict o modelo
pred.predictions.show()

Também podemos imprimir o coeficiente e a interceptação do modelo de regressão usando o seguinte comando:

In [ ]:
#coeficiente do modelo de regressão
coeff = regressor.coefficients

#X e Y intercept
intr = regressor.intercept

print ("O coeficiente do modelo é : %a" %coeff)
print ("O Intercept do modelo é : %f" %intr)


# Análise Estatística Básica

Assim que terminarmos com a operação de regressão linear básica, podemos ir um pouco mais além e analisar nosso modelo estatisticamente importando o módulo RegressionEvaluator do Pyspark.

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?